In [ ]:
#✅ Step 1: Install Required Packages
# Install required packages if not already installed
!pip install fastapi[all] sqlalchemy passlib[bcrypt] python-jose



In [9]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Boolean, DateTime, Float, UniqueConstraint
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from datetime import datetime

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    username = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_admin = Column(Boolean, default=False)
    reservations = relationship("Reservation", back_populates="user")

class Movie(Base):
    __tablename__ = 'movies'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    description = Column(String)
    genre = Column(String)
    poster_url = Column(String)
    showtimes = relationship("Showtime", back_populates="movie")

class Showtime(Base):
    __tablename__ = 'showtimes'
    id = Column(Integer, primary_key=True)
    movie_id = Column(Integer, ForeignKey('movies.id'))
    start_time = Column(DateTime)
    capacity = Column(Integer)
    movie = relationship("Movie", back_populates="showtimes")
    seats = relationship("Seat", back_populates="showtime")

class Seat(Base):
    __tablename__ = 'seats'
    id = Column(Integer, primary_key=True)
    showtime_id = Column(Integer, ForeignKey('showtimes.id'))
    seat_number = Column(String)
    is_reserved = Column(Boolean, default=False)
    reservation_id = Column(Integer, ForeignKey('reservations.id'), nullable=True)
    showtime = relationship("Showtime", back_populates="seats")
    reservation = relationship("Reservation", back_populates="seats")

    __table_args__ = (UniqueConstraint('showtime_id', 'seat_number', name='_showtime_seat_uc'),)

class Reservation(Base):
    __tablename__ = 'reservations'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    showtime_id = Column(Integer, ForeignKey('showtimes.id'))
    created_at = Column(DateTime, default=datetime.utcnow)
    user = relationship("User", back_populates="reservations")
    seats = relationship("Seat", back_populates="reservation")


In [10]:
#Step 3: Setup Database Session
# SQLite for demo; can swap to PostgreSQL/MySQL
DATABASE_URL = "sqlite:///./movie_reservation.db"

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create tables
Base.metadata.create_all(bind=engine)


In [11]:
#Step 4: Auth Utilities
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import timedelta

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

SECRET_KEY = "secret_key_for_demo"  # Replace in production
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

def hash_password(password: str):
    return pwd_context.hash(password)

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def create_access_token(data: dict, expires_delta: timedelta = None):
    to_encode = data.copy()
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(minutes=15)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt


In [15]:
pip install --user fastapi[all]



   ---------------------------------------- 0.0/719.9 kB ? eta -:--:--
   --------------------------------------- 719.9/719.9 kB 14.2 MB/s eta 0:00:00

   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   ----------------------------------------  0/17 [websockets]
   -- -------------------------------------  1/17 [sentry-sdk]
   -- -------------------------------------  1/17 [sentry-sdk]
   -- ----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [16]:
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from jose import jwt, JWTError
from sqlalchemy.orm import Session

app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def get_current_user(db: Session = Depends(get_db), token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None:
            raise credentials_exception
    except JWTError:
        raise credentials_exception
    user = db.query(User).filter(User.username == username).first()
    if user is None:
        raise credentials_exception
    return user


In [18]:
@app.post("/signup")
def signup(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = db.query(User).filter(User.username == form_data.username).first()
    if user:
        raise HTTPException(status_code=400, detail="Username already registered")
    hashed = hash_password(form_data.password)
    new_user = User(username=form_data.username, hashed_password=hashed)
    db.add(new_user)
    db.commit()
    db.refresh(new_user)
    return {"msg": "User created successfully"}

@app.post("/token")
def login(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = db.query(User).filter(User.username == form_data.username).first()
    if not user or not verify_password(form_data.password, user.hashed_password):
        raise HTTPException(status_code=400, detail="Invalid credentials")
    access_token = create_access_token(data={"sub": user.username})
    return {"access_token": access_token, "token_type": "bearer"}


In [19]:
from pydantic import BaseModel

class MovieCreate(BaseModel):
    title: str
    description: str
    genre: str
    poster_url: str

@app.post("/movies")
def create_movie(movie: MovieCreate, current_user: User = Depends(get_current_user), db: Session = Depends(get_db)):
    if not current_user.is_admin:
        raise HTTPException(status_code=403, detail="Not authorized")
    new_movie = Movie(**movie.dict())
    db.add(new_movie)
    db.commit()
    db.refresh(new_movie)
    return new_movie


In [20]:
db = SessionLocal()
admin = db.query(User).filter(User.username == "admin").first()
if not admin:
    admin = User(username="admin", hashed_password=hash_password("adminpass"), is_admin=True)
    db.add(admin)
    db.commit()
    print("Seed admin created: username=admin, password=adminpass")
else:
    print("Admin already exists.")
db.close()


Admin already exists.
